작성할 문서
- unet 생성
-텐서플로우 사용법
- 넘파이 사용법(무작위 배열 생성법, 넘파이 변환, 조작, 차원추)
-판다스, matplotlib사용법
-파이썬 코테 라이브러리


이미지 전처리

In [3]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

In [ ]:
path="C:/Users/82109/Desktop/datasets"

#해당 디렉토리의 모든 파일명을 가져옴

gifPath=[]
#경로 반환
def getFilePath(path,end=""):
  gifFileList=os.listdir(path)
  gifPath=[]
  for name in gifFileList:
    if name.endswith(".gif"):
      gifPath.append(os.path.join(path,name))
  return gifPath


#못쓰는 데이터를 걸러줌
def preprocessGif(path,frame=5):
  gif=Image.open(path)
  if gif.n_frames<frame:
    print(path,": ",gif.n_frames,"사용불가능")
    os.remove(path=path)
  else:
    print(path,": ",gif.n_frames," 사용가능")


gifPath=getFilePath(path) 

for i in gifPath:
  preprocessGif(i)


In [ ]:
#gif를 읽고 넘파이 배열로 노멀라이즈해줌
def loadGif(path, paddingSize=32):
  gif=Image.open(path)
  flip=np.random.randint(0,7,1)[0]

  images=[]
  for i in range(1,gif.n_frames):
    gif.seek(i)
    temp=gif.transpose(flip).convert("RGBA")
    temp=np.array(temp)
    print(temp.shape)
    height=(paddingSize-temp.shape[0]%paddingSize)%paddingSize
    width=(paddingSize-temp.shape[1]%paddingSize)%paddingSize
    temp=np.pad(temp, pad_width=((0,height),(0,width),(0,0)),mode="constant",constant_values=0)
    images.append(temp)
  gif.close()
  return np.array(images)/255.0

#인풋데이터와 아웃풋 데이터를 분리
def divide(arr):
  evens=arr[0::2]
  odds=arr[1::2]
  if evens.shape[0] != odds.shape[0]:
    evens=evens[0:-1]
  return [evens,odds]

arr=loadGif(gifPath[3])
input, output=divide(arr)
print(arr.shape ,input.shape, output.shape)

unet 모델 생성

In [25]:

def seperableConv(filter, input):
    depthwise=tf.keras.layers.Conv3D(input.shape[-1],3,padding="same",groups=input.shape[-1])(input)
    pointwise=tf.keras.layers.Conv3D(filter,1,padding="same")(depthwise)
    return pointwise
    
def block(filter,input):
    #conv1=tf.keras.layers.Conv3D(filter,3,padding="same")(input)
    conv1=seperableConv(filter,input)
    layerNorm1=tf.keras.layers.LayerNormalization()(conv1)
    swishAct1=tf.keras.layers.Activation("swish")(layerNorm1)

    #conv2=tf.keras.layers.Conv3D(filter,3,padding="same")(swishAct1)
    conv2=seperableConv(filter,swishAct1)
    layerNorm2=tf.keras.layers.LayerNormalization()(conv2)
    swishAct2=tf.keras.layers.Activation("swish")(layerNorm2)
    
    return swishAct2


def unetModel(inputShape=(None, None, None, 4)):
    inputImage=tf.keras.Input(shape=inputShape)
    inputRate=tf.keras.Input(shape=(inputShape[0],inputShape[1],inputShape[2],1))
    input=tf.keras.layers.Concatenate()([inputImage,inputRate])
    
    #인코딩
    e1=block(32,input)
    e1Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e1)
    
    e2=block(64,e1Pooling)
    e2Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e2)
    
    e3=block(128,e2Pooling)
    e3Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e3)
    
    e4=block(256,e3Pooling)
    e4Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e4)
    
    e5=block(512,e4Pooling)
    e5Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e5)
    
    #중간
    bottleNeck=block(1024,e5Pooling)

    d5UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(bottleNeck)
    d5Transpose=seperableConv(512,d5UpSampling)
    d5Concatenate=tf.keras.layers.Concatenate()([d5Transpose,e5])
    d5=block(512,d5Concatenate)
    
    d4UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d5)
    d4Transpose=seperableConv(256,d4UpSampling)
    d4Concatenate=tf.keras.layers.Concatenate()([d4Transpose,e4])
    d4=block(256,d4Concatenate)
    
    d3UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d4)
    d3Transpose=seperableConv(128,d3UpSampling)
    d3Concatenate=tf.keras.layers.Concatenate()([d3Transpose,e3])
    d3=block(128,d3Concatenate)
    
    d2UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d3)
    d2Transpose=seperableConv(64,d2UpSampling)
    d2Concatenate=tf.keras.layers.Concatenate()([d2Transpose,e2])
    d2=block(64,d2Concatenate)
    
    d1UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d2)
    d1Transpose=seperableConv(32,d1UpSampling)
    d1Concatenate=tf.keras.layers.Concatenate()([d1Transpose,e1])
    d1=block(32,d1Concatenate)
    
    outputImage=seperableConv(4,d1)
    
    return tf.keras.Model([inputImage,inputRate],outputImage)

unetModel((2,32,32,4)).summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 2, 32, 32,   0           []                               
                                4)]                                                               
                                                                                                  
 input_13 (InputLayer)          [(None, 2, 32, 32,   0           []                               
                                1)]                                                               
                                                                                                  
 concatenate_35 (Concatenate)   (None, 2, 32, 32, 5  0           ['input_12[0][0]',               
                                )                                 'input_13[0][0]']         

디퓨전 예시

In [30]:
class DiffusionModel(tf.keras.Model):
    def __init__(self,network:tf.keras.Model):
        super().__init__()
        self.network= network
        
        
    def train_step(self, images):
        # normalize images to have standard deviation of 1, like the noises
        images = self.normalizer(images, training=True)
        noises = tf.random.normal(shape=(batch_size, image_size, image_size, 3))

        # sample uniform random diffusion times
        diffusion_times = tf.random.uniform(
            shape=(1, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        noisy_images = signal_rates * images + noise_rates * noises

        with tf.GradientTape() as tape:
            # train the network to separate noisy images to their components
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, training=True
            )

            noise_loss = self.loss(noises, pred_noises)  # used for training
            image_loss = self.loss(images, pred_images)  # only used as metric

        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))

        self.noise_loss_tracker.update_state(noise_loss)
        self.image_loss_tracker.update_state(image_loss)

        # track the exponential moving averages of weights
        for weight, ema_weight in zip(self.network.weights, self.ema_network.weights):
            ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

        # KID is not measured during the training phase for computational efficiency
        return {m.name: m.result() for m in self.metrics[:-1]}

    def test_step(self, images):
        # normalize images to have standard deviation of 1, like the noises
        images = self.normalizer(images, training=False)
        noises = tf.random.normal(shape=(batch_size, image_size, image_size, 3))

        # sample uniform random diffusion times
        diffusion_times = tf.random.uniform(
            shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        noisy_images = signal_rates * images + noise_rates * noises

        # use the network to separate noisy images to their components
        pred_noises, pred_images = self.denoise(
            noisy_images, noise_rates, signal_rates, training=False
        )

        noise_loss = self.loss(noises, pred_noises)
        image_loss = self.loss(images, pred_images)

        self.image_loss_tracker.update_state(image_loss)
        self.noise_loss_tracker.update_state(noise_loss)

        # measure KID between real and generated images
        # this is computationally demanding, kid_diffusion_steps has to be small
        images = self.denormalize(images)
        generated_images = self.generate(
            num_images=batch_size, diffusion_steps=kid_diffusion_steps
        )
        self.kid.update_state(images, generated_images)

        return {m.name: m.result() for m in self.metrics}

In [31]:
network=unetModel(inputShape=(None,None,None,4))
model=DiffusionModel()
model.compile(
    tf.keras.optimizers.experimental.AdamW(
        learning_rate=1e-3, weight_decay=1e-4
    ),
    loss=tf.keras.losses.mean_squared_error,
              metrics=['accuracy'])

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.